# Brian demo

The code below shows a demonstration of Brian in action with a simple graphical interface. Run the cell below by clicking on it and hitting `CTRL+ENTER`. You can change the parameters of the model with the sliders, and immediately see the change in behaviour.

The model is a group of $N$ source neurons that fire Poisson spikes with a rate $R_\mathrm{max}(1+\sin(2\pi f t))/2$, connected via synapses to a target neuron. Each source neuron has a probability $p$ of being connected, and synapses have a certain fixed weight $w$. The target neuron has two variables, the membrane potential $V$ and the adaptation potential $V_t$ that evolve according to the differential equations:

$$\tau\frac{\mathrm{d}V}{\mathrm{dt}}=-V$$

$$\tau_t\frac{\mathrm{d}V_t}{\mathrm{dt}}=1-V_t$$

Each time the event $V>V_t$ occurs, the neuron spikes and $V$ is reset to 0, while $V_t$ increases by $\delta_t$.

This model is implemented with the following code:

```python
G = PoissonGroup(N, rates='R_max*0.5*(1+sin(2*pi*f*t))')
eqs = '''
dV/dt = -V/tau : 1
dVt/dt = (1-Vt)/tau_t : 1
'''
H = NeuronGroup(1, eqs, threshold='V>Vt', reset='V=0; Vt += delta_t', method='linear')
H.Vt = 1
S = Synapses(G, H, on_pre='V += w')
S.connect(p=p)
```

In [ ]:
%matplotlib inline
from brian2 import *
from brian2tools import *
import ipywidgets as ipw
prefs.codegen.target = 'numpy'

def model(N=20, R_max=150, f=10, w=0.1, p=0.5, tau=5, tau_t=50, delta_t=0.1):
    # Get parameters
    R_max = R_max*Hz
    f = f*Hz
    tau = tau*ms
    tau_t = tau_t*ms
    duration = 200*ms
    
    # Simulation code
    G = PoissonGroup(N, rates='R_max*0.5*(1+sin(2*pi*f*t))')
    eqs = '''
    dV/dt = -V/tau : 1
    dVt/dt = (1-Vt)/tau_t : 1
    '''
    H = NeuronGroup(1, eqs, threshold='V>Vt', reset='V=0; Vt += delta_t', method='linear')
    H.Vt = 1
    S = Synapses(G, H, on_pre='V += w')
    S.connect(p=p)
    # Run it
    MG = SpikeMonitor(G)
    MH = StateMonitor(H, ('V', 'Vt'), record=True)
    run(duration)
    
    # Plotting code
    figure(figsize=(15, 4))
    subplot(131)
    brian_plot(MG)
    title('Source neurons (Poisson)')
    subplot(132)
    plot(zeros(N), arange(N), 'ob')
    plot([0, 1], [S.i, ones(len(S.i))*N/2.], '-k')
    plot([1], [N/2.], 'og')
    xlim(-0.1, 1.1)
    ylim(-1, N)
    axis('off')
    title('Synapses')
    subplot(133)
    plot(MH.t, MH.V[0], label='V')
    plot(MH.t, MH.Vt[0], label='Vt')
    legend(loc='upper left')
    title('Target neuron')
    
layout = ipw.Layout(width='100%')
style = {'description_width': 'initial'}
ipw.interact(model,
             N=ipw.IntSlider(value=20, min=5, max=100, step=5, continuous_update=False,
                             description="Number of source neurons", style=style, layout=layout),
             R_max=ipw.FloatSlider(value=300, min=0, max=500, step=10, continuous_update=False,
                                   description=r"Source neuron max firing rate (Hz)", style=style, layout=layout),
             f=ipw.FloatSlider(value=10, min=1, max=50, step=1, continuous_update=False,
                               description=r"Source neuron frequency (Hz)", style=style, layout=layout),
             p=ipw.FloatSlider(value=0.5, min=0, max=1, step=0.01, continuous_update=False,
                               description=r"Synapse probability", style=style, layout=layout),
             w=ipw.FloatSlider(value=0.3, min=0, max=1, step=0.01, continuous_update=False,
                               description=r"Synapse weight", style=style, layout=layout),
             tau=ipw.FloatSlider(value=5, min=1, max=50, step=1, continuous_update=False,
                                 description=r"Target neuron membrane time constant (ms)", style=style, layout=layout),
             tau_t=ipw.FloatSlider(value=30, min=5, max=500, step=5, continuous_update=False,
                                   description=r"Target neuron adaptation constant (ms)", style=style, layout=layout),
             delta_t=ipw.FloatSlider(value=1.0, min=0, max=20, step=0.1, continuous_update=False,
                                     description=r"Target neuron adaptation strength", style=style, layout=layout),
            );